In [6]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda,RunnableBranch


load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [3]:
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human","Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human","Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)
escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

In [4]:
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

In [7]:
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | llm| StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | llm | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | llm | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | llm | StrOutputParser()
)

In [8]:
classification_chain = classification_template | llm | StrOutputParser()

In [9]:
chain = classification_chain | branches

* Run the chain with an example review
* Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
* Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
* Neutral review - "The product is okay. It works as expected but nothing exceptional."
* Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

In [12]:
review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

print(result)

Okay, here's a message escalating the feedback "Negative" to a human agent, designed to be concise and informative:

**Option 1 (Short & Sweet):**

> Escalating: Negative feedback received. Requires human review.

**Option 2 (Slightly More Detail):**

> Escalating: User feedback is "Negative." Please investigate.

**Option 3 (If you have a specific case ID or context):**

> Escalating: Feedback "Negative" related to [Case ID/Topic]. Requires human review.

Choose the option that best fits your system and provides the most relevant context for the human agent.  The key is to be clear that the feedback is negative and requires further attention.
